<a href="https://colab.research.google.com/github/malborroni/RECMojion/blob/master/Model_Classifier.pynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificatori tradizionali



Valutazione in 5-fold cross validation dei modelli
- feature extraction + SVC 
- feature extraction + Random Forest
- feature extraction + KNeighbors


**Introduzione**

Rete VGG face usata come feature extractor per un classificatore lineare. La speranza è che una rete trainata per riconoscere le facce sia piuttosto affine ad un task come il nostro di riconscimento di emozioni e dunque che con un taglio adatto si abbia una efficace face embedding, ovvero una rappresentazione ad altro livello di un volto, facllmente classificabile a questo punto da un classificatore classico

# Import Librerie

In [0]:
import keras
from keras.datasets import mnist 
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Model, Sequential, load_model
from keras.utils import to_categorical, np_utils 
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

from numpy import expand_dims,asarray
from PIL import Image
import random as rnd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('seaborn-pastel')

from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2 as cv
import os

# Dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pickle
with open('/content/gdrive/My Drive/AML/Prima parte/XY.pkl', 'rb') as infile:
    result = pickle.load(infile)
X,Y=result

# Preprocessing e Feature Extraction

In [0]:
import numpy as np
X = np.array(X)
Y = np.array(Y)

In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
le = LabelEncoder()
Y = le.fit_transform(Y)
#Y = to_categorical(Y, num_classes = 7)

In [0]:
! sudo pip install git+https://github.com/rcmalli/keras-vggface.git
# Example of face detection with a vggface2 model

import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

In [0]:
base_model = VGGFace(include_top = False, input_shape = (224, 224, 3), model='senet50')

In [0]:
X = preprocess_input( X.astype('float64'),version=2)

In [0]:
base_model.summary()

In [0]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer("global_average_pooling2d_13").output)

In [0]:
X = model.predict(X)

In [0]:
X.shape

(2590, 1024)

In [0]:
kfold_acc = {}
# dizionario per salvare la accuracy dei diversi modelli

# K fold


In [0]:
from sklearn.model_selection import StratifiedKFold
import numpy
# fix random seed for reproducibility
seed = 42
numpy.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)


## SVC

In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
clf = SVC(random_state=42)
accuracy = cross_val_score(clf, X, Y, scoring='accuracy', cv = kfold, n_jobs=-1)
print(accuracy)
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is: %.2f%% (+/- %.2f%%)"% (np.mean(accuracy), np.std(accuracy)))


[0.86486486 0.87451737 0.9034749  0.88030888 0.87451737]
Accuracy of Model with Cross Validation is: 0.88% (+/- 0.01%)


In [0]:
kfold_acc["SVC"]=(np.mean(accuracy), np.std(accuracy))

## Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, cross_val_score
clf = RandomForestClassifier(random_state=42)
accuracy = cross_val_score(clf, X, Y, scoring='accuracy', cv = kfold, n_jobs=-1)
print(accuracy)
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is: %.2f%% (+/- %.2f%%)"% (np.mean(accuracy), np.std(accuracy)))

[0.9034749  0.87644788 0.90540541 0.85328185 0.87451737]
Accuracy of Model with Cross Validation is: 0.88% (+/- 0.02%)


In [0]:
kfold_acc["RandomForest"]=(np.mean(accuracy), np.std(accuracy))

## KNeighbors

In [0]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(random_state=42)
accuracy = cross_val_score(neigh, X, Y, scoring='accuracy', cv = kfold, n_jobs=-1)
print(accuracy)
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is: %.2f%% (+/- %.2f%%)"% (np.mean(accuracy), np.std(accuracy)))

[0.85604607 0.89423077 0.84362934 0.90310078 0.86796117]
Accuracy of Model with Cross Validation is: 0.87% (+/- 0.02%)


In [0]:
kfold_acc["KNeighbors"]=(np.mean(accuracy), np.std(accuracy))

In [0]:
import json

json = json.dumps(kfold_acc)
f = open("/content/gdrive/My Drive/AML/kfold_acc.json","w")
f.write(json)
f.close()